# Fair LoRA Model Training

這個 notebook 使用公平性增強的 LoRA 微調方法 (Base: BAAI/bge-large-en-v1.5) 來進行履歷與職位描述配對，同時盡量降低不同教育背景群組之間的預測差異。

資料來源：現在已更新為重新分割後的資料集 `processed_resume_dataset_resplit`，確保在 train/val/test 中都存在正例，避免公平性指標退化。

## 目標

- 主任務：預測履歷與職位是否匹配 (Fit / No Fit)
- 敏感屬性：`is_top_school`, `school_category`
- 公平性策略：
  - 敏感屬性屏蔽（文字替換為中性標記）
  - LoRA 微調 (r=8, alpha=16, target_modules=['query','key','value'])
  - 對抗式去偏 (Adversarial Debiasing)
  - 多任務學習 (輔助預測 school_category 以正則化表示)
  - 加權/重抽樣減少群體不平衡
  - 公平性指標監控與早停 (Demographic Parity / Equalized Odds / Equal Opportunity)

## 流程
1. 環境與參數設置
2. 數據加載與敏感屬性屏蔽（使用新的重新分割資料集）
3. 建立模型與 LoRA 配置
4. 訓練迴圈（包含對抗 + 公平性損失）
5. 評估與公平性指標輸出（支援後續閾值微調）
6. 保存最佳模型與訓練記錄

執行 Cells 1→N 完成整體訓練。

In [8]:
# Imports & configuration (updated for new sampler/balanced validation)
import sys, json
from pathlib import Path

ROOT = Path('.').resolve()
sys.path.append(str(ROOT))

from fair_lora_config import (
    BASE_MODEL, DATASET_PATH, DEVICE, LEARNING_RATE, MAX_GRAD_NORM,
    ADVERSARIAL_LAMBDA, FAIRNESS_LAMBDA, MULTITASK_LAMBDA,
    USE_GROUP_BATCH_SAMPLER, USE_REWEIGHTING, CREATE_BALANCED_VAL,
    FAIRNESS_REG_LAMBDA, WINDOW_SELECTION_START, WINDOW_SELECTION_END,
    WINDOW_FAIRNESS_THRESHOLD, TEMPERATURE_CALIBRATION, AUC_ENABLED
)

from src.fair_data_loader import create_data_loaders
from src.fair_lora_model import FairLoRAModel
from src.fair_trainer import FairTrainer
import torch

print(f"Device: {DEVICE}")
print(f"Dataset path: {DATASET_PATH}")
print(f"GroupBatchSampler enabled: {USE_GROUP_BATCH_SAMPLER}")
print(f"Balanced validation enabled: {CREATE_BALANCED_VAL}")
print(f"Fairness regularizer lambda: {FAIRNESS_REG_LAMBDA}")
print(f"Temperature calibration: {TEMPERATURE_CALIBRATION}")
print(f"AUC metrics enabled: {AUC_ENABLED}")

Device: mps
Dataset path: /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/data/processed/processed_resume_dataset_resplit
GroupBatchSampler enabled: False
Balanced validation enabled: True
Fairness regularizer lambda: 0.05
Temperature calibration: True
AUC metrics enabled: True


In [9]:
# (Deprecated) Initial simple loader call replaced by enhanced call in next cell.
# Keeping this cell as a placeholder to avoid unpack errors.
print("Skip: use the enhanced create_data_loaders call in the next cell (returns 5 values).")

Skip: use the enhanced create_data_loaders call in the next cell (returns 5 values).


In [10]:
# Load data with new sampler + balanced validation loader
(
    train_loader,
    val_loader,
    test_loader,
    tokenizer,
    balanced_val_loader
) = create_data_loaders(
    dataset_path=str(DATASET_PATH),
    batch_size=8,  # reduced from 16 to 8 to mitigate MPS OOM
    max_seq_length=256,  # limit sequence length to 256 (try 384 if quality drops)
    use_reweighting=USE_REWEIGHTING and not USE_GROUP_BATCH_SAMPLER,  # avoid double fairness intervention
    use_group_batch_sampler=USE_GROUP_BATCH_SAMPLER,
    create_balanced_val=CREATE_BALANCED_VAL,
    num_workers=2 if DEVICE.type == 'mps' else 4,
    seed=42
)

print(f"Train batches: {len(train_loader)} | Val batches: {len(val_loader)} | Test batches: {len(test_loader)}")
print(f"Balanced Val loader: {'present' if balanced_val_loader is not None else 'none'}")


📦 Loading dataset from /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/data/processed/processed_resume_dataset_resplit...
   Train: 5,472 samples
   Val:   685 samples
   Test:  1,087 samples

📊 School Distribution:

   Train:
     no_school_mentioned...........   884 (16.15%)
     non_top_school................  4365 (79.77%)
     top_school....................   223 ( 4.08%)

   Val:
     non_top_school................   547 (79.85%)
     no_school_mentioned...........   111 (16.20%)
     top_school....................    27 ( 3.94%)

   Test:
     non_top_school................   867 (79.76%)
     no_school_mentioned...........   176 (16.19%)
     top_school....................    44 ( 4.05%)

🔤 Loading tokenizer: BAAI/bge-large-en-v1.5

⚖️  Using weighted sampling for fairness...

🧪 Balanced Val created (joint groups): 81 samples; per-group ~27

✅ Data loaders created successfully!
   Train batches: 684
   Val batches:   86
   Test batches:  136
   Balanced Val ba

In [11]:
# Build model with optional gradient checkpointing for memory savings
model = FairLoRAModel(
    base_model_name=BASE_MODEL,
    use_lora=True,
    use_adversarial=True,
    use_multitask=True,
    num_labels=2
)
model = model.to(DEVICE)

# Enable gradient checkpointing if available (helps reduce memory footprint)
try:
    base_attr = None
    for attr_name in ['base_model','model']:
        if hasattr(model, attr_name):
            candidate = getattr(model, attr_name)
            if hasattr(candidate, 'gradient_checkpointing_enable'):
                candidate.gradient_checkpointing_enable()
                base_attr = candidate
                break
    if base_attr is not None:
        print("Gradient checkpointing enabled.")
    else:
        print("Gradient checkpointing method not found; skipped.")
except Exception as e:
    print(f"Gradient checkpointing enabling failed: {e}")

🔧 Loading base model: BAAI/bge-large-en-v1.5
🔧 Applying LoRA (expanded targets, r=16, alpha=32)...
   • LoRA target modules detected: ['dense', 'key', 'query', 'value']
     (modules not present are ignored silently by PEFT)
🔒 Frozen encoder base weights except attention in last 4 layers; LoRA adapters remain trainable.
📊 Trainable params -> LoRA: 7,110,656 | Base last4(attn): 16,801,792 | Total: 23,912,448
trainable params: 23,912,448 || all params: 342,252,544 || trainable%: 6.9868
🔧 Adding adversarial discriminator...
🔧 Adding attribute classifier for multi-task learning...
🔧 Applying LoRA (expanded targets, r=16, alpha=32)...
   • LoRA target modules detected: ['dense', 'key', 'query', 'value']
     (modules not present are ignored silently by PEFT)
🔒 Frozen encoder base weights except attention in last 4 layers; LoRA adapters remain trainable.
📊 Trainable params -> LoRA: 7,110,656 | Base last4(attn): 16,801,792 | Total: 23,912,448
trainable params: 23,912,448 || all params: 342,25

In [12]:
# Build trainer with balanced val and fairness reg + temperature/AUC toggles
trainer = FairTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    balanced_val_loader=balanced_val_loader,
    device=str(DEVICE),
    learning_rate=LEARNING_RATE,
    adversarial_lambda=ADVERSARIAL_LAMBDA,
    fairness_lambda=FAIRNESS_LAMBDA,
    multitask_lambda=MULTITASK_LAMBDA,
    fairness_reg_lambda=FAIRNESS_REG_LAMBDA,
    max_grad_norm=MAX_GRAD_NORM,
    temperature_calibration=TEMPERATURE_CALIBRATION,
    auc_enabled=AUC_ENABLED
)
print("Trainer ready.")

Trainer ready.


In [13]:
# Scheduler + training loop params
num_epochs = 15
warmup_steps = int(0.1 * num_epochs * len(train_loader))  # 10% of total steps linear ramp
patience = 5

# CosineAnnealingWarmRestarts with restarts every ~2 epochs (measured in steps)
import math
steps_per_epoch = len(train_loader)
t0_epochs = 2
t0_steps = t0_epochs * steps_per_epoch  # since we step scheduler per batch
eta_min = 0.0  # minimum LR at cosine trough; can raise slightly if needed
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    trainer.main_optimizer,
    T_0=t0_steps,
    T_mult=2,  # progressively longer cycles
    eta_min=eta_min
)
print(f"Total steps: {num_epochs * steps_per_epoch}; Warmup steps (lambda ramp): {warmup_steps}")
print(f"Using CosineAnnealingWarmRestarts with T_0={t0_epochs} epochs (~{t0_steps} steps), T_mult=2.")

Total steps: 10260; Warmup steps (lambda ramp): 1026
Using CosineAnnealingWarmRestarts with T_0=2 epochs (~1368 steps), T_mult=2.


In [14]:
# Train with window selection and balanced fairness
history = trainer.train(
    num_epochs=num_epochs,
    early_stopping_patience=patience,
    scheduler=scheduler,
    warmup_steps=warmup_steps,
    adv_lambda_target=ADVERSARIAL_LAMBDA,
    multitask_lambda_target=MULTITASK_LAMBDA,
    fairness_lambda_target=FAIRNESS_LAMBDA,
    window_selection_start=WINDOW_SELECTION_START,
    window_selection_end=WINDOW_SELECTION_END,
    window_fairness_threshold=WINDOW_FAIRNESS_THRESHOLD
)
print("Training complete. Best epoch:", history.get('best_epoch'))
print("Window-best epoch:", history.get('window_best_epoch'))

Training on device: mps
Training batches: 684
Validation batches: 86
Adversarial debiasing: True
Multi-task learning: True
------------------------------------------------------------


Epoch 1 [Train]:   0%|          | 0/684 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warning

[Temperature Scaling] Calibrated temperature: 2.2736


Epoch 1 [Val-Balanced]:   0%|          | 0/11 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  w


Epoch 1/15
Train - Loss: 0.8339, Acc: 0.5577
Val   - Loss: 0.6955, Acc: 0.5109
Acc (thr=0.5 raw): 0.5314 | Acc (tuned): 0.5109
Fairness (thr=0.5) - Raw: 0.1741 | Balanced: 0.2275
Balanced Loader Fairness: 0.1763
                 - Demographic Parity Diff: 0.1632
                 - Equalized Odds Diff: 0.1656
Group positive rates (school_category): {0: {'count': 27, 'true_pos_rate': 0.48148148148148145, 'pred_pos_rate_fairness_thr': 0.37037037037037035, 'pred_pos_rate_tuned_thr': 1.0}, 1: {'count': 547, 'true_pos_rate': 0.5027422303473492, 'pred_pos_rate_fairness_thr': 0.25411334552102377, 'pred_pos_rate_tuned_thr': 0.9817184643510055}, 2: {'count': 111, 'true_pos_rate': 0.4954954954954955, 'pred_pos_rate_fairness_thr': 0.2072072072072072, 'pred_pos_rate_tuned_thr': 0.918918918918919}}
Group positive rates (is_top_school): {0: {'count': 658, 'true_pos_rate': 0.5015197568389058, 'pred_pos_rate_fairness_thr': 0.24620060790273557, 'pred_pos_rate_tuned_thr': 0.9711246200607903}, 1: {'count

...


Epoch 5/15
Train - Loss: 0.9191, Acc: 0.6234
Val   - Loss: 0.6916, Acc: 0.5226
Acc (thr=0.5 raw): 0.5679 | Acc (tuned): 0.5226
Fairness (thr=0.5) - Raw: 0.1178 | Balanced: 0.1328
Balanced Loader Fairness: 0.1507
                 - Demographic Parity Diff: 0.1300
                 - Equalized Odds Diff: 0.1550
Group positive rates (school_category): {0: {'count': 27, 'true_pos_rate': 0.48148148148148145, 'pred_pos_rate_fairness_thr': 0.4074074074074074, 'pred_pos_rate_tuned_thr': 1.0}, 1: {'count': 547, 'true_pos_rate': 0.5027422303473492, 'pred_pos_rate_fairness_thr': 0.5173674588665448, 'pred_pos_rate_tuned_thr': 0.9414990859232175}, 2: {'count': 111, 'true_pos_rate': 0.4954954954954955, 'pred_pos_rate_fairness_thr': 0.38738738738738737, 'pred_pos_rate_tuned_thr': 0.8828828828828829}}
Group positive rates (is_top_school): {0: {'count': 658, 'true_pos_rate': 0.5015197568389058, 'pred_pos_rate_fairness_thr': 0.49544072948328266, 'pred_pos_rate_tuned_thr': 0.9316109422492401}, 1: {'count

Epoch 6 [Train]:   0%|          | 0/684 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warning


Epoch 6/15
Train - Loss: 0.8194, Acc: 0.6319
Val   - Loss: 0.6893, Acc: 0.5182
Acc (thr=0.5 raw): 0.5650 | Acc (tuned): 0.5182
Fairness (thr=0.5) - Raw: 0.0951 | Balanced: 0.1503
Balanced Loader Fairness: 0.1519
                 - Demographic Parity Diff: 0.1441
                 - Equalized Odds Diff: 0.1466
Group positive rates (school_category): {0: {'count': 27, 'true_pos_rate': 0.48148148148148145, 'pred_pos_rate_fairness_thr': 0.5185185185185185, 'pred_pos_rate_tuned_thr': 1.0}, 1: {'count': 547, 'true_pos_rate': 0.5027422303473492, 'pred_pos_rate_fairness_thr': 0.6215722120658135, 'pred_pos_rate_tuned_thr': 0.9597806215722121}, 2: {'count': 111, 'true_pos_rate': 0.4954954954954955, 'pred_pos_rate_fairness_thr': 0.4774774774774775, 'pred_pos_rate_tuned_thr': 0.8918918918918919}}
Group positive rates (is_top_school): {0: {'count': 658, 'true_pos_rate': 0.5015197568389058, 'pred_pos_rate_fairness_thr': 0.5972644376899696, 'pred_pos_rate_tuned_thr': 0.9483282674772037}, 1: {'count':

Epoch 7 [Train]:   0%|          | 0/684 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warning


Epoch 7/15
Train - Loss: 1.0107, Acc: 0.6193
Val   - Loss: 0.7115, Acc: 0.5328
Acc (thr=0.5 raw): 0.5591 | Acc (tuned): 0.5328
Fairness (thr=0.5) - Raw: 0.1608 | Balanced: 0.1643
Balanced Loader Fairness: 0.1925
                 - Demographic Parity Diff: 0.1091
                 - Equalized Odds Diff: 0.1439
Group positive rates (school_category): {0: {'count': 27, 'true_pos_rate': 0.48148148148148145, 'pred_pos_rate_fairness_thr': 0.37037037037037035, 'pred_pos_rate_tuned_thr': 1.0}, 1: {'count': 547, 'true_pos_rate': 0.5027422303473492, 'pred_pos_rate_fairness_thr': 0.263254113345521, 'pred_pos_rate_tuned_thr': 0.9542961608775137}, 2: {'count': 111, 'true_pos_rate': 0.4954954954954955, 'pred_pos_rate_fairness_thr': 0.26126126126126126, 'pred_pos_rate_tuned_thr': 0.9009009009009009}}
Group positive rates (is_top_school): {0: {'count': 658, 'true_pos_rate': 0.5015197568389058, 'pred_pos_rate_fairness_thr': 0.2629179331306991, 'pred_pos_rate_tuned_thr': 0.9452887537993921}, 1: {'count'

In [ ]:
# Inspect metrics CSV and fairness trajectories
import pandas as pd
from pathlib import Path
metrics_csv = Path('models/fair_adversarial/epoch_metrics.csv')
if metrics_csv.exists():
    df_metrics = pd.read_csv(metrics_csv)
    display(df_metrics.tail())
    print("Fairness raw (last 5):", df_metrics['fairness_score_raw'].tail().tolist())
    print("Fairness balanced (last 5):", df_metrics['fairness_score_balanced'].tail().tolist())
else:
    print("Metrics CSV not found yet.")

Artifacts saved to models/fair_adversarial


In [ ]:
# Evaluate on test with best and window-best checkpoints
import torch
from pathlib import Path

best_epoch = history.get('best_epoch')
window_best_epoch = history.get('window_best_epoch')

print(f"Best epoch: {best_epoch}; Window-best epoch: {window_best_epoch}")

# Load window-best if exists
window_ckpt = Path('models/fair_adversarial/window_best_model.pt')
if window_ckpt.exists():
    trainer.load_checkpoint(window_ckpt)
    print("Loaded window-best checkpoint for test evaluation.")
else:
    print("Window-best checkpoint not found; using current trainer state.")

# Simple test evaluation using stored best threshold
best_thr_list = history.get('best_thresholds', [])
if best_thr_list and best_epoch:
    thr_idx = max(0, best_epoch-1)
    best_thr = best_thr_list[thr_idx]
else:
    best_thr = 0.5

model.eval()
labels_all = []
probs_all = []
sc_all = []
ts_all = []
with torch.no_grad():
    for batch in test_loader:
        ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        labels = batch['label'].to(DEVICE)
        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        logits = out['logits']
        probs = torch.softmax(logits, dim=1)[:,1]
        labels_all.extend(labels.cpu().numpy())
        probs_all.extend(probs.cpu().numpy())
        sc_all.extend(batch['school_category'].cpu().numpy())
        ts_all.extend(batch['is_top_school'].cpu().numpy())

import numpy as np
labels_np = np.array(labels_all)
probs_np = np.array(probs_all)
preds_np = (probs_np >= best_thr).astype(int)
acc_test = (preds_np == labels_np).mean() if labels_np.size else 0.0
print(f"Test accuracy (threshold={best_thr:.3f}): {acc_test:.4f}")

MPS matmul ok: torch.Size([2, 4])


In [ ]:
# Summary of training history
import pandas as pd
import json
from pathlib import Path

print("History keys:", list(history.keys()))
print("Best epoch:", history.get('best_epoch'))
print("Window-best epoch:", history.get('window_best_epoch'))
print("Temperature trajectory:", history.get('temperature')[-5:])
print("Adv lambda final per epoch:", history.get('adv_lambdas')[-5:])
print("Multitask lambda final per epoch:", history.get('multitask_lambdas')[-5:])
print("Fairness reg gaps last 5:", history.get('fairness_reg_gaps')[-5:])

metrics_csv = Path('models/fair_adversarial/epoch_metrics.csv')
if metrics_csv.exists():
    df_metrics = pd.read_csv(metrics_csv)
    print("\nAUC last 5:", df_metrics['val_auc'].tail().tolist())
    print("PR-AUC last 5:", df_metrics['val_pr_auc'].tail().tolist())
else:
    print("Metrics CSV not found.")

,epoch,best_threshold,val_loss,val_acc,fairness_score
0,1,0.05,0.691992,0.519708,0.006883
1,2,0.05,0.690551,0.531387,0.006883
2,3,0.05,0.687889,0.562044,0.006883
3,4,0.05,0.686120,0.563504,0.006883
4,5,0.05,0.684175,0.556204,0.006883


Best epoch: 5
Best epoch threshold: 0.05
